In [1]:
from __future__ import print_function
import pickle
import os.path
import base64
import email
from apiclient import errors
import re
import time
from datetime import datetime
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

def main():
    """Shows basic usage of the Gmail API.
    Lists the user's Gmail labels.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('gmail', 'v1', credentials=creds)
    
    return service

#     # Call the Gmail API
#     results = service.users().labels().list(userId='me').execute()
#     labels = results.get('labels', [])
    
#     userInfo = service.users().getProfile(userId='me').execute()
#     print("User Info :",userInfo)

#     if not labels:
#         print('No labels found.')
#     else:
#         print('Labels:')
#         for label in labels:
#             print(label['name'])





# if __name__ == '__main__':
#     main()

main()

In [2]:
results=main().users().labels().list(userId='me').execute()
label=results.get('labels', [])
[i['name'] for i in label]

['CHAT',
 'SENT',
 'INBOX',
 'IMPORTANT',
 'TRASH',
 'DRAFT',
 'SPAM',
 'CATEGORY_FORUMS',
 'CATEGORY_UPDATES',
 'CATEGORY_PERSONAL',
 'CATEGORY_PROMOTIONS',
 'CATEGORY_SOCIAL',
 'STARRED',
 'UNREAD']

In [8]:
results = main().users().messages().list(userId='me',labelIds = ['INBOX'],maxResults=500).execute()

In [9]:
messages = results.get('messages', [])

In [10]:
while 'nextPageToken' in results:
    page_token=results['nextPageToken']
#     print(page_token)
    results = main().users().messages().list(userId='me',labelIds = ['INBOX'],maxResults=500,pageToken=page_token).execute()
    messages.extend(results['messages'])


In [11]:
len([i['id'] for i in messages])

26070

In [12]:
start=time.time()
print(start)
string_check= re.compile('[@_!#$%^&*()<>?/\|}{~]')
msg_read=[]
date=[]
ff=[]
sub=[]
text=[]
length=[]
id=[]
time_taken=[]
for i in range(15): # You can put:len([i['id'] for i in messages]) instead of 15 in range
    if i%200==0:
        print((datetime.now()).strftime('%H:%M:%S'))
        time_taken.append((datetime.now()).strftime('%H:%M:%S'))
    print(i)
    print(messages[i]['id'])
    id.append(messages[i]['id'])
    msg=main().users().messages().get(userId='me',id=messages[i]['id'],format="full").execute()
    subject=[i['value'] for i in msg['payload']['headers'] if i['name']=='Subject' or i['name']=='SUBJECT'][0]
    From=[i['value'] for i in msg['payload']['headers'] if i['name']=='from' or i['name']=='FROM' or i['name']=='From'][0]
    Date=[i['value'] for i in msg['payload']['headers'] if i['name']=='Date' or i['name']=='DATE'][0]
    msg_read.append(msg['snippet'])
    date.append(Date)
    ff.append(From)
    sub.append(subject)
    msg=main().users().messages().get(userId='me',id=messages[i]['id'],format="raw").execute()
    try:
        msg_str=base64.urlsafe_b64decode(msg['raw']).decode()
    except:
        msg_str=base64.urlsafe_b64decode(msg['raw']).decode(encoding='windows-1252')
    mime_msg=email.message_from_string(msg_str)
    
    length.append(len(mime_msg.get_payload()))
    
    
    a=[]
    a1=[]
    if mime_msg.is_multipart():
        
        for payload in mime_msg.get_payload():
            a.append(payload.get_payload())
            
            while type(a[0])==list:
                a=[i.get_payload() for i in a[0]]
            
            
#         if type(a1[0])==list:
#             b=mime_msg.get_payload()[0]
            
#             for i in b.get_payload():
#                 a.append(i.get_payload())
                
#         else:
#             for payload in mime_msg.get_payload():
#                 a.append(payload.get_payload())
            
        
# #         for payload in mime_msg.get_payload():
# #             a.append(payload.get_payload())
    else:
        a.append(mime_msg.get_payload())
    
    t1=a[0].replace("\r"," ").replace("\n"," ")
    text.append(" ".join([i for i in t1.split() if (string_check.search(i)==None) and ('http' not in i)]))
    
    print("Subject: ",subject)
#     print("From: ",From)
#     print("Date: ",Date)
#     print(msg['snippet'])
    print("      Next        ")
    
print("time taken",time.time()-start) 
taken=time.time()-start

1588519869.2685912
21:01:09
0
171da967b2e3009a
Subject:  Important Message about Scrum Hyderabad
      Next        
1
171da8240bc539de
Subject:  [Analytics Vidhya] Weekly Progress Reminder
      Next        
2
171d9b87843b689c
Subject:  Require a Machine Learning Developer - Full Time/Freelancer at Tryambaka Techno Solutions and 3 more data scientist jobs in Bengaluru, Karnataka for you!
      Next        
3
171d98f789b16942
Subject:  📢 Launching Emotional First Aid
      Next        
4
171d95aa31d6a63f
Subject:  Don't forget: 2 events this Saturday
      Next        
5
171d91fc750b69a7
Subject:  ATM Withdrawal Alert
      Next        
6
171d91f2f1293f5c
Subject:  ATM Withdrawal Alert
      Next        
7
171d87bfdfed320b
Subject:  Will the lockdown restrictions in India extend until September?
      Next        
8
171d8665f70faf3e
Subject:  We need your support!
      Next        
9
171d85e855ceec1b
Subject:  Google Colab: How does it compare to a GPU-enabled laptop? | Dario Radečić i

In [13]:
f=[" ".join(i.split()[:-1]) if len(i.split())>1 else i for i in ff]

In [14]:
import pandas as pd
df=pd.DataFrame(list(zip(id,date,ff,sub,msg_read,text)),columns=['Mail_Id','Date','From','Subject','Short Text','Body'])
df.head()

,Mail_Id,Date,From,Subject,Short Text,Body
0,171da967b2e3009a,"Sun, 03 May 2020 12:50:25 +0000 (UTC)",Meetup <info@meetup.com>,Important Message about Scrum Hyderabad,Meetup Your Meetup Group will shut down soon! ...,"Members of Scrum Hyderabad, Your Organizer, Ag..."
1,171da8240bc539de,"Sun, 03 May 2020 12:28:20 +0000",Analytics Vidhya <noreply@notify.thinkific.com>,[Analytics Vidhya] Weekly Progress Reminder,Analytics Vidhya ‌ Hi virender singh tanwar! T...,screen and .main-table width: 300px margin: 0 ...
2,171d9b87843b689c,"Sun, 3 May 2020 03:47:56 -0500 (CDT)",Indeed <alert@indeed.com>,Require a Machine Learning Developer - Full Ti...,Sampoorna Computer People and Tryambaka Techno...,Indeed Job Alert 4 new data scientist vacancie...
3,171d98f789b16942,"Sun, 3 May 2020 08:03:10 +0000","""mind.fit"" <marketing@curefit.com>",📢 Launching Emotional First Aid,Something on your mind? Let&#39;s tackle it to...,"html PUBLIC XHTML 1.0 Transitional xmlns:v=3D""..."
4,171d95aa31d6a63f,"Sun, 03 May 2020 07:05:25 +0000 (UTC)",Go Adventure - Hyd <info@meetup.com>,Don't forget: 2 events this Saturday,Meetup Event reminders Go Adventure - Hyd Satu...,only screen and .other-meetups-tearsheet displ...


In [ ]:
df.to_csv(r'C:\Users\lenovo\Documents\Project\my mail.csv',index=None)

In [15]:
import os
os

<module 'os' from 'C:\\Users\\lenovo\\Anaconda3\\lib\\os.py'>

In [16]:
os.listdir()

['.anaconda',
 '.bash_history',
 '.conda',
 '.condarc',
 '.config',
 '.gitconfig',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.keras',
 '.matplotlib',
 '.spyder-py3',
 '3D Objects',
 'Anaconda3',
 'AppData',
 'Application Data',
 'Contacts',
 'Cookies',
 'credentials.json',
 'Deleting the Mail Using GMAIL API.ipynb',
 'Desktop',
 'Documents',
 'Downloads',
 'Dropbox',
 'Favorites',
 'GMAIL API 2.ipynb',
 'Gmail API.ipynb',
 'IntelGraphicsProfiles',
 'Links',
 'Local Settings',
 'M5 Kaggle.ipynb',
 'MicrosoftEdgeBackups',
 'Music',
 'My Documents',
 'NetHood',
 'NTUSER.DAT',
 'ntuser.dat.LOG1',
 'ntuser.dat.LOG2',
 'NTUSER.DAT{0057ead2-43ee-11ea-af50-50e0857e7822}.TM.blf',
 'NTUSER.DAT{0057ead2-43ee-11ea-af50-50e0857e7822}.TMContainer00000000000000000001.regtrans-ms',
 'NTUSER.DAT{0057ead2-43ee-11ea-af50-50e0857e7822}.TMContainer00000000000000000002.regtrans-ms',
 'ntuser.ini',
 'OneDrive',
 'Pictures',
 'PrintHood',
 'Python Practice.ipynb',
 'Reading Email using GMAIL API.ipyn